In [1]:
!pip install pandas
!pip install opencv-python
!apt-get install -y libgl1
!pip install kaggle
!pip install scipy


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1 is already the newest version (1.4.0-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import h5py
import shutil
import cv2
import random
import uuid
import numpy as np

2024-08-11 22:41:14.942577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 22:41:14.959579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 22:41:14.964621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 22:41:14.980266: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range = 0.1,
    shear_range=0.5,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.3
)

In [5]:

train_image_gen = image_gen.flow_from_directory(os.path.join("new_training_data"),
                                               target_size=(130,130),
                                                color_mode='rgb',
                                               batch_size=16,
                                               class_mode='binary',subset="training")
val_image_gen = image_gen.flow_from_directory(os.path.join("new_training_data"),target_size=(130,130),color_mode='rgb',batch_size=16,class_mode='binary',subset='validation')

Found 45560 images belonging to 2 classes.
Found 19524 images belonging to 2 classes.


In [6]:
model = tf.keras.models.load_model("./isic-model-3.keras")

I0000 00:00:1723416078.480935    6024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723416078.487846    6024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723416078.487897    6024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723416078.489188    6024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723416078.489232    6024 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [7]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 130, 130, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 130, 130, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     1,188,726 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,269,924 (237.54 MB)

 Trainable params: 18,746,742 (71.51 MB)

 Non-trainable params: 6,029,696 (23.00 MB)

 Optimizer params: 37,493,486 (143.03 MB)

In [8]:

model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=0.75,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction='sum_over_batch_size',
    name='binary_focal_crossentropy'
),metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives()])

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=3)
model.fit(train_image_gen,epochs=50, callbacks=[early_stop],validation_data=val_image_gen,batch_size=16)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-11 22:42:00.260467: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8
2024-08-11 22:42:03.786900: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2024-08-11 22:42:04.417484: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:1723416124.477315    6150 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416124.520351    6150 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduce

   1/2848 ━━━━━━━━━━━━━━━━━━━━ 32:37:01 41s/step - auc: 0.8095 - binary_accuracy: 0.6875 - false_negatives: 1.0000 - false_positives: 4.0000 - loss: 0.0983 - recall: 0.8571

W0000 00:00:1723416133.209661    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.213851    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.217769    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.221863    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.225861    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.229857    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.234627    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.238434    6152 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723416133.242334    6152 gp

 426/2848 ━━━━━━━━━━━━━━━━━━━━ 1:18:42 2s/step - auc: 0.8118 - binary_accuracy: 0.6924 - false_negatives: 219.5516 - false_positives: 847.3826 - loss: 0.0683 - recall: 0.8532

...

2024-08-12 10:34:49.416769: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-08-12 10:34:58.222193: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


2848/2848 ━━━━━━━━━━━━━━━━━━━━ 8861s 3s/step - auc: 0.9096 - binary_accuracy: 0.7651 - false_negatives: 407.2116 - false_positives: 4982.9282 - loss: 0.0421 - recall: 0.9627 - val_auc: 0.9417 - val_binary_accuracy: 0.7801 - val_false_negatives: 121.0000 - val_false_positives: 4172.0000 - val_loss: 0.0341 - val_recall: 0.9876
Epoch 7/50


2024-08-12 13:02:30.117197: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 5 of 8
2024-08-12 13:02:38.307142: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


2704/2848 ━━━━━━━━━━━━━━━━━━━━ 5:01 2s/step - auc: 0.9145 - binary_accuracy: 0.7696 - false_negatives: 300.3610 - false_positives: 4791.8027 - loss: 0.0395 - recall: 0.9728

KeyboardInterrupt: 

In [10]:
model.save('isic-model-7.keras')